In [1]:
# !pip install imutils
# !pip install -U torchvision
# !pip install torchsummary
# !pip install opencv-python
# !pip install matplotlib
# !pip install scikit-image


In [2]:
import time
import cv2
import os
import random
import numpy as np
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
from PIL import Image
import imutils
from math import *
import random
import xml.etree.ElementTree as ET 
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms.functional as TF
from torchvision import datasets, models, transforms
from torch.utils.data import Dataset


ModuleNotFoundError: No module named 'imutils'

In [ ]:
print(torchvision.__version__)

<span style="font-size:25px;">**Check if GPU is available**</span>

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

<span style="font-size:25px;">**Display example data**</span>

<span style="font-size:25px;">**Data preprocessing**</span>

To prevent the neural network from overfitting the training dataset, the dataset is transoformed randomly. The following operations are applied to the training and validation dataset:
* Since the face occupies a very small portion of the entire image, crop the image and use only the face for training.
* Resize the cropped face into a (224x224) image.
* Randomly change the brightness and saturation of the resized face.
* Randomly rotate the face after the above three transformations.
* Convert the image and landmarks into torch tensors and normalize them between [-1, 1].

In [ ]:
class Transforms():
    def __init__(self):
        pass
    
    def crop_face(self, image, landmarks, crops):
        top = int(crops['top'])
        left = int(crops['left'])
        height = int(crops['height'])
        width = int(crops['width'])

        image = TF.crop(image, top, left, height, width)

        img_shape = np.array(image).shape
        landmarks = torch.tensor(landmarks) - torch.tensor([[left, top]])
        landmarks = landmarks / torch.tensor([img_shape[1], img_shape[0]])
        return image, landmarks
    
    def resize(self, image, landmarks, img_size):
        image = TF.resize(image, img_size)
        return image, landmarks
    
    def color_jitter(self, image, landmarks):
        #ranNum = random.random()
        color_jitter = transforms.ColorJitter(brightness=random.random(), 
                                              contrast=random.random(),
                                              saturation=random.random(), 
                                              hue=random.uniform(0,0.5))
        image = color_jitter(image)
        return image, landmarks
    
    def rotate(self, image, landmarks, angle):
        angle = random.uniform(-angle, +angle)

        transformation_matrix = torch.tensor([
            [+cos(radians(angle)), -sin(radians(angle))], 
            [+sin(radians(angle)), +cos(radians(angle))]
        ])

        image = imutils.rotate(np.array(image), angle)

        landmarks = landmarks - 0.5
        new_landmarks = np.matmul(landmarks, transformation_matrix)
        new_landmarks = new_landmarks + 0.5
        return Image.fromarray(image), new_landmarks
    
    def __call__(self, image, landmarks, crops):
        image = Image.fromarray(image)
        image, landmarks = self.crop_face(image, landmarks, crops)
        image, landmarks = self.resize(image, landmarks, (224, 224))
        image, landmarks = self.color_jitter(image, landmarks)
        image, landmarks = self.rotate(image, landmarks, angle=random.randint(-30,30))
        
        image = TF.to_tensor(image)
        image = TF.normalize(image, [0.5], [0.5])
        return image, landmarks

<span style="font-size:25px;">**Dataset Preparation**</span>

The labels_ibug_300W_train.xml contains the image path, landmarks and coordinates for the bounding box. Store these values in lists to access them easily during training.

In [ ]:
class FaceLandmarksDataset(Dataset):

    def __init__(self, transform=None):
        tree = ET.parse('../data/ibug_300W_large_face_landmark_dataset/labels_ibug_300W_train.xml')
        root = tree.getroot()

        self.image_filenames = []
        self.landmarks = []
        self.crops = []
        self.transform = transform
        self.root_dir = '../data/ibug_300W_large_face_landmark_dataset'
        
        for filename in root[2]:
            self.image_filenames.append(os.path.join(self.root_dir, filename.attrib['file']))

            self.crops.append(filename[0].attrib)

            landmark = []
            for num in range(68):
                x_coordinate = int(filename[0][num].attrib['x'])
                y_coordinate = int(filename[0][num].attrib['y'])
                landmark.append([x_coordinate, y_coordinate])
            self.landmarks.append(landmark)

        self.landmarks = np.array(self.landmarks).astype('float32')     

        assert len(self.image_filenames) == len(self.landmarks)

    def __len__(self):
        return len(self.image_filenames)

    def __getitem__(self, index):
        image = cv2.imread(self.image_filenames[index], 0)
        landmarks = self.landmarks[index]
        
        if self.transform:
            image, landmarks = self.transform(image, landmarks, self.crops[index])

        landmarks = landmarks - 0.5

        return image, landmarks

In [ ]:
OriDataset = FaceLandmarksDataset()
image, landmarks = OriDataset[150]
plt.figure(figsize=(10, 10))
plt.imshow(image, cmap='gray');
plt.scatter(landmarks[:,0], landmarks[:,1], s=40);

<span style="font-size:25px;">**Example of image after pre-processing**</span>

In [1]:
dataset = FaceLandmarksDataset(Transforms())

image, landmarks = dataset[5]
landmarks = (landmarks + 0.5) * 224
plt.figure(figsize=(10, 10))
plt.imshow(image.numpy().squeeze(), cmap='gray');
plt.scatter(landmarks[:,0], landmarks[:,1], s=40);

NameError: name 'FaceLandmarksDataset' is not defined

In [ ]:
# Train-Test split
len_test_set = int(0.2*len(dataset))
len_train_set = len(dataset) - len_test_set 

print("The length of Train set is {}".format(len_train_set))
print("The length of Test set is {}".format(len_test_set))

train_dataset , test_dataset = torch.utils.data.random_split(dataset , [len_train_set, len_test_set])

# setting batch sizes and shuffle the data
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=6)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=True, num_workers=6)

In [ ]:
images, landmarks = next(iter(train_loader))
#torch.set_printoptions(profile="full")
#print(images.shape)
#print(landmarks.shape)
print(f"Images batch shape: {images.size()}")
print(f"Landmarks batch shape: {landmarks.size()}")

In [ ]:
images, landmarks = next(iter(test_loader))
print(f"Images batch shape: {images.size()}")
print(f"Landmarks batch shape: {landmarks.size()}")

In [ ]:
class ResNet50(nn.Module):
    def __init__(self,num_classes=136):
        super().__init__()
        self.model_name='resnet50'
        self.model=models.resnet50()
        self.model.conv1=nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.model.fc=nn.Linear(self.model.fc.in_features, num_classes)
        
    def forward(self, x):
        x=self.model(x)
        return x

In [ ]:
class ResNet101(nn.Module):
    def __init__(self,num_classes=136):
        super().__init__()
        self.model_name='resnet101'
        self.model=models.resnet101()
        self.model.conv1=nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.model.fc=nn.Linear(self.model.fc.in_features, num_classes)
        
    def forward(self, x):
        x=self.model(x)
        return x

In [ ]:
class Inception(nn.Module):
    def __init__(self,num_classes=136):
        super().__init__()
        self.model_name='inceptionv3'
        self.model=models.inception_v3()
        self.model.Conv2d_1a_3x3.conv=nn.Conv2d(1, 32, kernel_size=3, stride=2, bias=False)
        self.model.AuxLogits.fc = nn.Linear(self.model.AuxLogits.fc.in_features, num_classes)
        self.model.fc=nn.Linear(self.model.fc.in_features, num_classes)
        
    def forward(self, x):
        x=self.model(x)
        return x

In [ ]:
class DenseNet(nn.Module):
    def __init__(self,num_classes=136):
        super().__init__()
        self.model_name='densenet'
        self.model = models.densenet161()
        self.model.features.conv0=nn.Conv2d(1, 96, kernel_size=7, stride=2, padding=3, bias=False)
        self.model.classifier = nn.Linear(self.model.classifier.in_features, num_classes)
        
    def forward(self, x):
        x=self.model(x)
        return x

In [ ]:
class EfficientNet(nn.Module):
    def __init__(self,num_classes=136):
        super().__init__()
        self.model_name='efficientnet'
        self.model = models.efficientnet_b4()
        self.model.features[0][0]=nn.Conv2d(1, 48, kernel_size=3, stride=2, padding=1, bias=False)
        self.model.classifier[1]= nn.Linear(self.model.classifier[1].in_features, num_classes)
        
        # Khởi tạo lại running_mean và running_var của lớp BatchNorm2d
        self.model.features[0][1].reset_running_stats() 
        
    def forward(self, x):
        x=self.model(x)
        return x

In [ ]:
class ConvNeXt(nn.Module):
    def __init__(self,num_classes=136):
        super().__init__()
        self.model_name='convnext'
        self.model = models.convnext_base()
        self.model.features[0][0]=nn.Conv2d(1, 128, kernel_size=4, stride=4, bias=False)
        self.model.classifier[2]= nn.Linear(self.model.classifier[2].in_features, num_classes)
        
    def forward(self, x):
        x=self.model(x)
        return x

In [ ]:
import sys

def print_overwrite(step, total_step, loss, operation):
    sys.stdout.write('\r')
    if operation == 'train':
        sys.stdout.write("Train Steps: %d/%d  Loss: %.4f " % (step, total_step, loss))   
    else:
        sys.stdout.write("Valid Steps: %d/%d  Loss: %.4f " % (step, total_step, loss))
        
    sys.stdout.flush()

##### if os.path.isdir('progress'):
    !rm -rf progress
os.mkdir('progress')

In [ ]:
model1 = ResNet50()
model2 = ResNet101()
# model3 = Inception()
# model4 = DenseNet()
# model5 = EfficientNet()
model6 = ConvNeXt()

In [ ]:
def euclidean_distance(predictions, landmarks):
    """
    Tính khoảng cách Euclidean giữa các landmark dự đoán và thực tế.

    Args:
      predictions: Các landmark dự đoán, có dạng (batch_size, 68, 2).
      landmarks: Các landmark thực tế, có dạng (batch_size, 68, 2).

    Returns:
      Khoảng cách Euclidean trung bình giữa các landmark.
    """
    return torch.mean(torch.sqrt(torch.sum((predictions - landmarks) ** 2, dim=2)))

def train_test(model, epochs, learning_rate, train_loader, test_loader):
    torch.autograd.set_detect_anomaly(True)
    network = model
    # network.load_state_dict("checkpoint/progress2.pth")
    network.load_state_dict(torch.load("checkpoint/progress.pth"))  # Đúng
    network.to(device)    

    criterion = nn.MSELoss()
    optimizer = optim.Adam(network.parameters(), lr=learning_rate)
    num_epochs = epochs
    loss_min = np.inf
    
    train_loss_record = []
    test_loss_record = []

    start_time = time.time()
    for epoch in range(1,num_epochs+1):

        loss_train = 0
        loss_test = 0
        running_loss = 0
        
        train_accuracy = 0
        test_accuracy = 0

        network.train()
        for step in range(1,len(train_loader)+1):

            images, landmarks = next(iter(train_loader))

            images = images.to(device)
            landmarks = landmarks.view(landmarks.size(0),-1).to(device) 

            predictions = network(images)

            # clear all the gradients before calculating them
            optimizer.zero_grad()

            # find the loss for the current step
            loss_train_step = criterion(predictions,landmarks)

            #loss_valid_step = criterion(predictions.logits, landmarks)

            # calculate the gradients
            loss_train_step.backward()

            # update the parameters
            optimizer.step()

            loss_train = loss_train + loss_train_step.item()
            running_loss = loss_train/step

            print_overwrite(step, len(train_loader), running_loss, 'train')
            
            # Tính toán độ chính xác trên tập huấn luyện
            predictions = (predictions.view(-1, 68, 2).cpu() + 0.5) * 224
            landmarks = (landmarks.view(-1, 68, 2).cpu() + 0.5) * 224
            train_accuracy += euclidean_distance(predictions, landmarks)

        network.eval() 
        with torch.no_grad():

            for step in range(1,len(test_loader)+1):

                images, landmarks = next(iter(test_loader))

                images = images.to(device)
                landmarks = landmarks.view(landmarks.size(0),-1).to(device)

                predictions = network(images)

                # find the loss for the current step
                loss_test_step = criterion(predictions, landmarks)
                #loss_valid_step = criterion(predictions.logits, landmarks)

                loss_test = loss_test + loss_test_step.item()
                running_loss = loss_test/step

                print_overwrite(step, len(test_loader), running_loss, 'test')
                
                # Tính toán độ chính xác trên tập kiểm tra
                predictions = (predictions.view(-1, 68, 2).cpu() + 0.5) * 224
                landmarks = (landmarks.view(-1, 68, 2).cpu() + 0.5) * 224
                test_accuracy += euclidean_distance(predictions, landmarks)

        loss_train = loss_train / len(train_loader)
        loss_test = loss_test / len(test_loader)
        
        train_accuracy = train_accuracy / len(train_loader)
        test_accuracy = test_accuracy / len(test_loader)

        print('\n--------------------------------------------------')
        print('Epoch: {}  Train Loss: {:.6f}  Test Loss: {:.6f}'.format(epoch, loss_train, loss_test))
        print('Train Accuracy: {:.6f}  Test Accuracy: {:.6f}'.format(train_accuracy, test_accuracy))
        print('--------------------------------------------------')
        
        train_loss_record.append(loss_train)
        test_loss_record.append(loss_test)

        if loss_test < loss_min:
            loss_min = loss_test
            torch.save(network.state_dict(), 'checkpoint/progress.pth')  
            print("\nMinimum Test Loss of {:.6f} at epoch {}/{}".format(loss_min, epoch, num_epochs))
            print('Model Saved\n')

    print('Training Complete')
    print("Total Elapsed Time : {} s".format(time.time()-start_time))

    
    plt.subplot(1,2,1)
    plt.title("Train Loss")
    plt.plot(train_loss_record)
    plt.xticks(range(1,len(train_loss_record)+1, 1))
    plt.ylabel('MSE Loss')
    plt.xlabel('Epochs')
    
    plt.subplot(1,2,2)
    plt.title("Test Loss")
    plt.plot(test_loss_record)
    plt.xticks(range(1,len(test_loss_record)+1, 1))
    plt.ylabel('MSE Loss')
    plt.xlabel('Epochs')

In [ ]:
def show_result(model, test_loader):
    start_time = time.time()

    with torch.no_grad():

        best_network = model
        best_network.to(device)
        best_network.load_state_dict(torch.load('/home/phong/PycharmProjects/Filter project/notebook/checkpoint/progress2.pth')) 
        best_network.eval()
 
        images, landmarks = next(iter(test_loader))

        images = images.to(device)
        landmarks = (landmarks + 0.5) * 224   

        predictions = (best_network(images).cpu() + 0.5) * 224
        predictions = predictions.view(-1,68,2)

        plt.figure(figsize=(10,40))

        for img_num in range(10):
            plt.subplot(1,10,img_num+1)
            plt.imshow(images[img_num].cpu().numpy().transpose(1,2,0).squeeze(), cmap='gray')
            plt.scatter(predictions[img_num,:,0], predictions[img_num,:,1], c = 'r', s = 5)
            plt.scatter(landmarks[img_num,:,0], landmarks[img_num,:,1], c = 'g', s = 5)

    print('Total number of test images: {}'.format(len(test_dataset)))

    end_time = time.time()
    print("Elapsed Time : {}".format(end_time - start_time)) 

In [ ]:
# train_test(model1,60, 0.0001,train_loader,test_loader)

In [ ]:
# show_result(model1, test_loader)

In [ ]:
# import datetime
# 
# best_network = ResNet50()
# best_network.load_state_dict(torch.load('checkpoint/progress.pth'))
# best_network = best_network.to(device)  # Chuyển mô hình lên GPU
# best_network.eval()
# 
# # Đọc ảnh mới
# image_path = 'test9.jpg'
# image = cv2.imread(image_path, 1)  # Đọc ảnh xám
# plt.imshow(image)
# # Phát hiện khuôn mặt (ví dụ sử dụng OpenCV)
# face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
# faces = face_cascade.detectMultiScale(image,1.1, 4)
# 
# # Tiền xử lý ảnh
# for (x, y, w, h) in faces:
#     face_image_gray = cv2.cvtColor(image[y:y+h, x:x+w], cv2.COLOR_BGR2GRAY)  # Chuyển sang ảnh xám
#     face_image = Image.fromarray(face_image_gray)
#     face_image = transforms.Resize((224, 224))(face_image)
#     face_image = transforms.ToTensor()(face_image)
#     face_image = transforms.Normalize([0.5], [0.5])(face_image)
#     face_image = face_image.unsqueeze(0).to(device)  # Thêm chiều batch
# 
#     # Dự đoán landmark
#     with torch.no_grad():
#         predictions = best_network(face_image)
#     predictions = (predictions.cpu() + 0.5) * 224
#     predictions = predictions.view(-1, 68, 2).numpy()
#     
#     scale_x = w / 224  # w là chiều rộng của bounding box
#     scale_y = h / 224  # h là chiều cao của bounding box
#         
#         # Chuyển đổi tọa độ landmark về ảnh gốc
#     for i in range(68):
#         landmark_x = int(predictions[0, i, 0] * scale_x + x)  # x là tọa độ x của bounding box
#         landmark_y = int(predictions[0, i, 1] * scale_y + y)  # y là tọa độ y của bounding box
#     
#         # Vẽ landmark lên ảnh gốc
#         cv2.circle(image, (landmark_x, landmark_y), 2, (0, 255, 0), -1)
#     # Vẽ landmark lên ảnh gốc
#     # for i in range(68):
#     #     cv2.circle(image, (int(x + predictions[0, i, 0]), int(y + predictions[0, i, 1])), 2, (0, 255, 0), -1)
#     cv2.rectangle(image, (x, y), (x+w, y+h), (255, 0, 0), 2)    
# 
# current_datetime = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
# 
# # Tạo thư mục results nếu chưa tồn tại
# results_dir = 'Results'
# if not os.path.exists(results_dir):
#     os.makedirs(results_dir)
# 
# # Lưu kết quả với tên file là ngày giờ
# output_filename = f"{current_datetime}.jpg"
# output_path = os.path.join(results_dir, output_filename)
# cv2.imwrite(output_path, image)
# # Hiển thị kết quả
# cv2.imshow('Result', image)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [ ]:
# import cv2
# from PIL import Image
# import torch
# from torchvision import transforms
# import numpy as np
# 
# # ... (Các hàm và lớp đã được định nghĩa trước đó, bao gồm ResNet50, Transforms, ...) ...
# 
# # Thiết bị (GPU nếu có)
# device = "cuda" if torch.cuda.is_available() else "cpu"
# 
# # Nạp mô hình đã huấn luyện
# best_network = ResNet50()  # Hoặc kiến trúc mạng tương ứng
# best_network.load_state_dict(torch.load('checkpoint/trainRS50_120.pth'))
# best_network.to(device)
# best_network.eval()
# 
# # Khởi tạo camera
# cap = cv2.VideoCapture(0)
# 
# # Nạp mô hình phát hiện khuôn mặt Haar Cascade
# face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
# 
# while(True):
#     # Đọc một khung hình từ camera
#     ret, frame = cap.read()
# 
#     # Phát hiện khuôn mặt
#     faces = face_cascade.detectMultiScale(frame, scaleFactor=1.3, minNeighbors=8)
# 
#     # Tiền xử lý ảnh và dự đoán landmark cho từng khuôn mặt
#     for (x, y, w, h) in faces:
#         face_image_gray = cv2.cvtColor(frame[y:y+h, x:x+w], cv2.COLOR_BGR2GRAY)  # Chuyển sang ảnh xám
#         face_image = Image.fromarray(face_image_gray)
#         face_image = transforms.Resize((224, 224))(face_image)
#         face_image = transforms.ToTensor()(face_image)
#         face_image = transforms.Normalize([0.5], [0.5])(face_image)
#         face_image = face_image.unsqueeze(0).to(device)  # Thêm chiều batch
# 
#         # Dự đoán landmark
#         with torch.no_grad():
#             predictions = best_network(face_image)
#         predictions = (predictions.cpu() + 0.5) * 224
#         predictions = predictions.view(-1, 68, 2).numpy()
# 
#         # Vẽ landmark lên ảnh gốc (ảnh màu)
#         # for i in range(68):
#         #     cv2.circle(frame, (int(x + predictions[0, i, 0]), int(y + predictions[0, i, 1])), 2, (0, 255, 0), -1)
#         # ... (Các phần khác của code) ...
# 
# # Tính toán tỷ lệ resize
#         scale_x = w / 224  # w là chiều rộng của bounding box
#         scale_y = h / 224  # h là chiều cao của bounding box
#         
#         # Chuyển đổi tọa độ landmark về ảnh gốc
#         for i in range(68):
#             landmark_x = int(predictions[0, i, 0] * scale_x + x)  # x là tọa độ x của bounding box
#             landmark_y = int(predictions[0, i, 1] * scale_y + y)  # y là tọa độ y của bounding box
#         
#             # Vẽ landmark lên ảnh gốc
#             cv2.circle(frame, (landmark_x, landmark_y), 2, (0, 255, 0), -1)
# 
#         
#         # Vẽ filter (ví dụ: vẽ mắt kính)
#         left_eye_center = (int(predictions[0, 36, 0] + x), int(predictions[0, 36, 1] + y))
#         right_eye_center = (int(predictions[0, 45, 0] + x), int(predictions[0, 45, 1] + y))
#         eye_distance = int(np.linalg.norm(np.array(left_eye_center) - np.array(right_eye_center)))
#         distance_factor = 1 + (eye_distance / frame.shape[1])  # Tỷ lệ khoảng cách mắt với chiều rộng khung hình
#         glasses_width = int(eye_distance * 1.5 * distance_factor)  # Tăng hệ số nhân
#         glasses_height = int(glasses_width * 0.4)
#         glasses_x = int(left_eye_center[0] - glasses_width * 0.25)
#         glasses_y = int(left_eye_center[1] - glasses_height * 0.5)
# 
#         # cv2.rectangle(frame, (glasses_x, glasses_y), (glasses_x + glasses_width, glasses_y + glasses_height), (0, 0, 255), 2)
# 
#     # Hiển thị khung hình
#     cv2.imshow('Realtime Filter', frame)
#     # Thoát khi nhấn phím 'q'
#     if cv2.waitKey(1) & 0xFF == ord('q'):
#         break
# 
# # Giải phóng camera và đóng cửa sổ
# cap.release()
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [ ]:
import cv2
from PIL import Image
import torch
from torchvision import transforms
import numpy as np
import os
import datetime
import matplotlib.pyplot as plt 

# ... (Các hàm và lớp đã được định nghĩa trước đó, bao gồm ResNet50, Transforms, ...) ...

# Thiết bị (GPU nếu có)
device = "cuda" if torch.cuda.is_available() else "cpu"

# Nạp mô hình đã huấn luyện
best_network = ResNet50()  # Hoặc kiến trúc mạng tương ứng
best_network.load_state_dict(torch.load('checkpoint/trainRS50_120.pth'))
best_network.to(device)
best_network.eval()

# Nạp mô hình phát hiện khuôn mặt Haar Cascade
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

def detect_and_draw_landmarks_realtime():
    """
    Phát hiện khuôn mặt và vẽ landmark trong thời gian thực từ camera.
    """
    cap = cv2.VideoCapture(0)

    while(True):
        # Đọc một khung hình từ camera
        ret, frame = cap.read()

        # Phát hiện khuôn mặt
        faces = face_cascade.detectMultiScale(frame, scaleFactor=1.3, minNeighbors=8)

        # Tiền xử lý ảnh và dự đoán landmark cho từng khuôn mặt
        for (x, y, w, h) in faces:
            face_image_gray = cv2.cvtColor(frame[y:y+h, x:x+w], cv2.COLOR_BGR2GRAY)
            face_image = Image.fromarray(face_image_gray)
            face_image = transforms.Resize((224, 224))(face_image)
            face_image = transforms.ToTensor()(face_image)
            face_image = transforms.Normalize([0.5], [0.5])(face_image)
            face_image = face_image.unsqueeze(0).to(device)

            # Dự đoán landmark
            with torch.no_grad():
                predictions = best_network(face_image)
            predictions = (predictions.cpu() + 0.5) * 224
            predictions = predictions.view(-1, 68, 2).numpy()

            # Tính toán tỷ lệ resize
            scale_x = w / 224
            scale_y = h / 224
            
            # Chuyển đổi tọa độ landmark về ảnh gốc
            for i in range(68):
                landmark_x = int(predictions[0, i, 0] * scale_x + x)
                landmark_y = int(predictions[0, i, 1] * scale_y + y)
            
                # Vẽ landmark lên ảnh gốc
                cv2.circle(frame, (landmark_x, landmark_y), 2, (0, 255, 0), -1)

        # Hiển thị khung hình
        cv2.imshow('Realtime Filter', frame)
        # Thoát khi nhấn phím 'q'
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Giải phóng camera và đóng cửa sổ
    cap.release()
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
def detect_and_draw_landmarks_image(image_path):
    """
    Phát hiện khuôn mặt và vẽ landmark từ một ảnh.
    """
    # Đọc ảnh mới
    image = cv2.imread(image_path, 1)

    # Phát hiện khuôn mặt
    faces = face_cascade.detectMultiScale(image,1.1, 4)

    # Tiền xử lý ảnh
    for (x, y, w, h) in faces:
        face_image_gray = cv2.cvtColor(image[y:y+h, x:x+w], cv2.COLOR_BGR2GRAY)
        face_image = Image.fromarray(face_image_gray)
        face_image = transforms.Resize((224, 224))(face_image)
        face_image = transforms.ToTensor()(face_image)
        face_image = transforms.Normalize([0.5], [0.5])(face_image)
        face_image = face_image.unsqueeze(0).to(device)

        # Dự đoán landmark
        with torch.no_grad():
            predictions = best_network(face_image)
        predictions = (predictions.cpu() + 0.5) * 224
        predictions = predictions.view(-1, 68, 2).numpy()
        
        scale_x = w / 224
        scale_y = h / 224
            
        # Chuyển đổi tọa độ landmark về ảnh gốc
        for i in range(68):
            landmark_x = int(predictions[0, i, 0] * scale_x + x)
            landmark_y = int(predictions[0, i, 1] * scale_y + y)
        
            # Vẽ landmark lên ảnh gốc
            cv2.circle(image, (landmark_x, landmark_y), 2, (0, 255, 0), -1)

        cv2.rectangle(image, (x, y), (x+w, y+h), (255, 0, 0), 2)    

    current_datetime = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")

    # Tạo thư mục results nếu chưa tồn tại
    results_dir = 'Results'
    if not os.path.exists(results_dir):
        os.makedirs(results_dir)

    # Lưu kết quả với tên file là ngày giờ
    output_filename = f"{current_datetime}.jpg"
    output_path = os.path.join(results_dir, output_filename)
    cv2.imwrite(output_path, image)
    # Hiển thị kết quả
    cv2.imshow('Result', image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# Gọi hàm để chạy
# detect_and_draw_landmarks_realtime()  # Chạy với camera
detect_and_draw_landmarks_image('test3.jpg')  # Chạy với ảnh